In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import BCELoss, MSELoss, L1Loss
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [2]:
path = '/workspace/GNCA_tumor/biopharma_activity.csv'
# data = pd.read_csv(path , on_bad_lines='skip')
data = pd.read_csv(path , error_bad_lines=False) #IF ABOVE LINE GIVES ERROR UNCOMMENT THIS LINE AND COMMENT OUT ABOVE LINE AND THEN RUN
data = data.dropna(subset=['Smiles','Standard Value','Molecular Weight','AlogP','Standard Units'])
data.reset_index(drop=True, inplace=True)
data.head()


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type
0,CHEMBL205876,NaN,None,361.83,0,4.99,9,O=C(Nc1ccc(Cl)cc1)c1ccccc1Cn1ccc2cccnc21,IC50,>',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1139022,1,Scientific Literature,Bioorg Med Chem Lett,2006.0,None,NaN,NaN
1,CHEMBL201090,NaN,None,406.39,0,3.39,16a,COc1ccc2[nH]c3c4c(OC)c(OC)c(OC)cc4c4c(c3c2c1)C...,Inhibition,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1149451,1,Scientific Literature,J Med Chem,2006.0,CHEMBL3308860,NaN,NaN
2,CHEMBL202196,NaN,None,338.37,0,3.89,20,O=C1NC(=O)C(c2c[nH]c3ccccc23)=C1c1cccc2ccccc12,Inhibition,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1149451,1,Scientific Literature,J Med Chem,2006.0,CHEMBL3308860,NaN,NaN
3,CHEMBL201463,NaN,None,392.41,0,3.07,18,COc1cc(C2=C(c3c(C)[nH]c4ccccc34)C(=O)NC2=O)cc(...,Inhibition,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1149451,1,Scientific Literature,J Med Chem,2006.0,CHEMBL3308860,NaN,NaN
4,CHEMBL372944,NaN,None,339.35,0,2.28,5,COc1cc(C2=C(c3ccccc3)C(=O)NC2=O)cc(OC)c1OC,Inhibition,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1149451,1,Scientific Literature,J Med Chem,2006.0,CHEMBL3308860,NaN,NaN


In [3]:
unique_units = data['Standard Units'].unique()  #finding different units of solubility present in our data
print(unique_units)

#we can only use nM, %, uM, ug.mL-1, we drop all other values 
units_to_drop = ['%', 'hr',  'ug.mL-1', "10'-4No_unit", 'degrees C', "10'-3/s",
                 "10'5/M/s", '% ID/g', 'equiv']
data = data[~data["Standard Units"].isin(units_to_drop)]
data.reset_index(drop=True, inplace=True)
data['Standard Value'] = data.apply(lambda row:
                                    row['Standard Value'] * 1000 if row['Standard Units'] in ('µM','uM') else row['Standard Value'],axis=1)

data["Standard Units"] = 'nM'

mean_value = data["Standard Value"].mean()
std_dev = data["Standard Value"].std()

# Perform Z-score normalization
data["Standard Value"] = (data["Standard Value"] - mean_value) / std_dev

['nM' '%' 'hr' 'uM' 'ug.mL-1' "10'-4No_unit" 'degrees C' "10'-3/s"
 "10'5/M/s" '% ID/g' 'µM' 'equiv']


In [4]:
data

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type
0,CHEMBL205876,NaN,None,361.83,0,4.99,9,O=C(Nc1ccc(Cl)cc1)c1ccccc1Cn1ccc2cccnc21,IC50,>',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1139022,1,Scientific Literature,Bioorg Med Chem Lett,2006.0,None,NaN,NaN
1,CHEMBL201511,MALEIMIDE,None,378.38,0,2.76,10,COc1cc(C2=C(c3c[nH]c4ccccc34)C(=O)NC2=O)cc(OC)...,IC50,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1149451,1,Scientific Literature,J Med Chem,2006.0,None,NaN,NaN
2,CHEMBL382478,NaN,None,404.47,1,5.35,36,O=C(Nc1ccc(-c2ccccc2)cc1)c1ccccc1Cn1ccc2ncnc-2c1,IC50,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1139022,1,Scientific Literature,Bioorg Med Chem Lett,2006.0,None,NaN,NaN
3,CHEMBL410903,NaN,None,442.52,1,5.87,37,COc1cc2nccc(Oc3ccc4c(C(=O)NC5CCCC5)cccc4c3)c2c...,IC50,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1240342,1,Scientific Literature,J Mol Graph Model,2009.0,None,NaN,NaN
4,CHEMBL1241777,NaN,None,444.49,0,4.77,59,COCCNC(=O)c1cccc2cc(Oc3ccnc4cc(OC)c(C(C)=O)cc3...,IC50,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1240342,1,Scientific Literature,J Mol Graph Model,2009.0,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15523,CHEMBL98,VORINOSTAT,4,264.32,0,2.47,1; SAHA,O=C(CCCCCCC(=O)Nc1ccccc1)NO,IC50,>',...,Homo sapiens,SINGLE PROTEIN,CHEMBL5214883,1,Scientific Literature,Eur J Med Chem,2021.0,None,NaN,NaN
15524,CHEMBL24828,VANDETANIB,4,475.36,0,4.43,Vandetanib,COc1cc2/c(=N/c3ccc(Br)cc3F)nc[nH]c2cc1OCC1CCN(...,IC50,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL5214883,1,Scientific Literature,Eur J Med Chem,2021.0,None,NaN,INHIBITOR
15525,CHEMBL5174514,NaN,None,569.68,2,5.31,8,Cc1cnc(C#Cc2ccc3c4c(oc3c2)C(C)(C)c2cc(NS(C)(=O...,IC50,>',...,Homo sapiens,SINGLE PROTEIN,CHEMBL5113522,1,Scientific Literature,J Med Chem,2022.0,None,NaN,NaN
15526,CHEMBL5218990,NaN,None,595.47,2,7.39,60,COc1cc2ncnc(Sc3cccc(NC(=S)Nc4ccc(Br)c(C(F)(F)F...,IC50,=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL5214889,1,Scientific Literature,Eur J Med Chem,2021.0,None,TIME = 0.1667 hr,INHIBITOR


In [5]:
smiles_list = data['Smiles']
smiles_list

0                 O=C(Nc1ccc(Cl)cc1)c1ccccc1Cn1ccc2cccnc21
1        COc1cc(C2=C(c3c[nH]c4ccccc34)C(=O)NC2=O)cc(OC)...
2         O=C(Nc1ccc(-c2ccccc2)cc1)c1ccccc1Cn1ccc2ncnc-2c1
3        COc1cc2nccc(Oc3ccc4c(C(=O)NC5CCCC5)cccc4c3)c2c...
4        COCCNC(=O)c1cccc2cc(Oc3ccnc4cc(OC)c(C(C)=O)cc3...
                               ...                        
15523                          O=C(CCCCCCC(=O)Nc1ccccc1)NO
15524    COc1cc2/c(=N/c3ccc(Br)cc3F)nc[nH]c2cc1OCC1CCN(...
15525    Cc1cnc(C#Cc2ccc3c4c(oc3c2)C(C)(C)c2cc(NS(C)(=O...
15526    COc1cc2ncnc(Sc3cccc(NC(=S)Nc4ccc(Br)c(C(F)(F)F...
15527    CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)...
Name: Smiles, Length: 15528, dtype: object

In [6]:
SMILES_CHARS = [' ',
                '#', '%', '(', ')', '+', '-', '.', '/',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                '=', '@',
                'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                'R', 'S', 'T', 'V', 'X', 'Z',
                '[', '\\', ']',
                'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                't', 'u']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )
def smiles_encoder(compound, maxlen=295):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles(compound))
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c] ] = 1
    return X

one_hot_list = []
for i in range(len(smiles_list)):
    mat=smiles_encoder(smiles_list[i])
    one_hot_list.append(mat)

In [7]:
X = one_hot_list


In [54]:
X[1].shape

(295, 56)

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        # self.labels = labels  
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dataset = CustomDataset(X)

batch_size = 32  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [9]:
class VAE(nn.Module):
    def __init__(self , input_size, latent_size):
        super(VAE, self).__init__()
        self.input_size = input_size
        # The encoder part of the VAE
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc21 = nn.Linear(128, latent_size) 
        self.fc22 = nn.Linear(128, latent_size) 
        
        # The decoder part of the VAE
        self.fc3 = nn.Linear(latent_size, 128) 
        self.fc4 = nn.Linear(128, input_size) 
    
    def encode(self, x):
        # Encode the image
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def reparameterize(self, mu, logvar):
        # Sample from the latent space
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        # Decode the latent vector back to an image
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        # Forward pass through the VAE
        mu, logvar = self.encode(x.view(-1, self.input_size))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [43]:
def vae_loss(recon_x, x, mu, logvar):
    # Binary cross-entropy loss
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    # Kullback-Leibler divergence loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())   
    # Return the total loss
    return BCE + KLD

In [44]:
model = VAE(295*56,20)
optimizer = optim.Adam(model.parameters(),lr=0.004, weight_decay=0.05)
epochs = 20

In [45]:
for epoch in range(epochs):
    train_loss = 0
    with tqdm(train_loader, unit="batch") as tepoch: 
        for data in tepoch:
            
            optimizer.zero_grad()
            inputs = data.to(torch.float32)
            recon_inputs, mu, logvar = model(inputs)
            loss = vae_loss(recon_inputs, inputs, mu, logvar)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Print the average loss for each epoch
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, train_loss/len(train_loader.dataset)))

  0%|          | 0/486 [00:00<?, ?batch/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Using a target size (torch.Size([32, 295, 56])) that is different to the input size (torch.Size([32, 16520])) is deprecated. Please ensure they have the same size.
  This is separate from the ipykernel package so we can avoid doing imports until
100%|█████████▉| 485/486 [07:28<00:01,  1.08s/batch]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Using a target size (torch.Size([8, 295, 56])) that is different to the input size (torch.Size([8, 16520])) is deprecated. Please ensure they have the same size.
  This is separate from the ipykernel package so we can avoid doing imports until
  0%|          | 0/486 [00:00<?, ?batch/s]

Epoch [1/20], Loss: 350.7067


  0%|          | 0/486 [00:00<?, ?batch/s]

Epoch [2/20], Loss: 174.4402


  0%|          | 0/486 [00:00<?, ?batch/s]

...

Epoch [18/20], Loss: 127.2643


  0%|          | 0/486 [00:00<?, ?batch/s]

Epoch [19/20], Loss: 126.5804


100%|██████████| 486/486 [07:25<00:00,  1.09batch/s]

Epoch [20/20], Loss: 126.4032


In [52]:
def check_valid(mol):
    if mol == '':
        return False

    mol = Chem.MolFromSmiles(mol, sanitize=True)
    if mol is None:
        return False
    return True
def is_unique(mol, generated):
    if mol in generated: return False
    return True

In [61]:
def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[ i ]
    return smi

In [66]:
list_of_mols=[]
for i in range(100):
    latent_sample = torch.randn(1, 20)  

    with torch.no_grad():
        generated_data = model.decode(latent_sample)

    generated_data = generated_data.reshape(295,56)
    molecule = np.zeros((295,56))
    molecule_list=[]
    for n,i in enumerate(generated_data):
        idx = i.argmax().numpy()
        molecule[n][idx]=1
        molecule_list.append(list(molecule[n]))  
    molecule=molecule.astype(np.int8)
    smile = smiles_decoder(molecule)
    list_of_mols.append(smile)
    print(smile)
    print("   ")

CNc1ncOc2ccccnc3cc3c(c((CcNNc)cccc2)cF((C)C)(4)cc)C)c(1)c112)c114ssoPCC1nn1C.FFnlnF2FF\sc()(=O)cC55ccT5b(1gBc25))1)OBr434(O#31oNcOc1)1(=O%1NCCCCNe7#1C4l+HK3V0T+%IIlH-F\L88SR/4O0]iLV6cR(lgVi(LgZ.ZnHsIe3C]KX =5gsZK+ p /6(+oB=23#ZB@K6IX862]T#=t@t#51LTC4.b52itNPsoocK7#a@ iIiAr4S/RA\[rKIsV8 lcTi[2Ms
   
Cc1ncn(]cccc(((C)Nc)c2c(nc3cc3cccc(Brc)c(Ccccn)cCc4)cn2)c233F)CC)F)11C3H]l1[(N@H])[[H3])c3)cH]3c1cccccccCcC(c16sccc46)cc5)3()C(COC)cc2)c11%7=N/O84]+%R0IpcZ93X#T9u.(uIaScLgVKi-MNo[(aZT\Tn)]R%7NCZbZF2g5PSerMB/ S)(RT)bba4HF@MZ@L)FTAX=SNc(41(CgnZc2TM86.0@VrLnu2XAaS%7+[]P1\g5Z#5tBT06P%MHXn)s+L7c9uo8
   
O=CCCCc(Nc2cccNc(cc)ccclCcccccccccccc1nCCcO))c)c)c1))c1)c1)C[1H]]Br6lrp-(Osn4(5OBc)c=cc))c4cc)Nc=[5+]cC(-c(Br)6c)cc(Br)c4O(13)=N/O)cc2Br)c1)=N/O84V+%RVI4AS9eH#T9u3(+IM#cLKVK#-]PR[(tAraT70]RnsNCZbZF2gTZ/9rNBp S)(] )b3aoHFLM2s5)S%AX=aNF(-1/Rgn#c29T96.Bn1[lnuFHL OV7+[KP1a.AZ#5a8(0)P%MHKBessI0c9po8
   
Cc1ccccccn(=l)cc2)(1C(=()cc(ccccnNC)cF)c1)cc1)c11l1sAZ.\.Zl##/N-Ipt45nL3l[2[-44Nn)CFCH]CC4[24[C([2H]

In [67]:
validity=0
for molecule in list_of_mols:
    validity += int(check_valid(molecule))
print(validity)

0


RDKit ERROR: [07:24:29] SMILES Parse Error: syntax error while parsing: CNc1ncOc2ccccnc3cc3c(c((CcNNc)cccc2)cF((C)C)(4)cc)C)c(1)c112)c114ssoPCC1nn1C.FFnlnF2FF\sc()(=O)cC55ccT5b(1gBc25))1)OBr434(O#31oNcOc1)1(=O%1NCCCCNe7#1C4l+HK3V0T+%IIlH-F\L88SR/4O0]iLV6cR(lgVi(LgZ.ZnHsIe3C]KX
RDKit ERROR: [07:24:29] SMILES Parse Error: Failed parsing SMILES 'CNc1ncOc2ccccnc3cc3c(c((CcNNc)cccc2)cF((C)C)(4)cc)C)c(1)c112)c114ssoPCC1nn1C.FFnlnF2FF\sc()(=O)cC55ccT5b(1gBc25))1)OBr434(O#31oNcOc1)1(=O%1NCCCCNe7#1C4l+HK3V0T+%IIlH-F\L88SR/4O0]iLV6cR(lgVi(LgZ.ZnHsIe3C]KX' for input: 'CNc1ncOc2ccccnc3cc3c(c((CcNNc)cccc2)cF((C)C)(4)cc)C)c(1)c112)c114ssoPCC1nn1C.FFnlnF2FF\sc()(=O)cC55ccT5b(1gBc25))1)OBr434(O#31oNcOc1)1(=O%1NCCCCNe7#1C4l+HK3V0T+%IIlH-F\L88SR/4O0]iLV6cR(lgVi(LgZ.ZnHsIe3C]KX'
RDKit ERROR: [07:24:29] SMILES Parse Error: syntax error while parsing: Cc1ncn(]cccc(((C)Nc)c2c(nc3cc3cccc(Brc)c(Ccccn)cCc4)cn2)c233F)CC)F)11C3H]l1[(N@H])[[H3])c3)cH]3c1cccccccCcC(c16sccc46)cc5)3()C(COC)cc2)c11%7=N/O84]+%R0IpcZ93